In [33]:
# Libraries used:
# pdfplumber
# re
# pandas
# collection

In [1]:
#pip3 install pdfplumber

In [2]:
import pdfplumber
import re

In [3]:
# Example 1 -  a small  invoice
with pdfplumber.open('Dataset/professionalblue_example.pdf') as pdf:
    page = pdf.pages[0]
    text = page.extract_text()

In [4]:
text

'INVOICE\nInvoice No. I1083\nAccount # C1006\nDate 08-14-2008\nDue By 08-31-2008\nDemo Company\nPhone : 111-222-3333 Terms None\n1234 Main Street E-Mail : 333-444-4444 PO No. PO1234\nAshland, KY 41102 Web : http://www.ksoftware.net Sales Rep SalesPerson1\nBill To Ship To\nTest Customer Test Customer\n1234 Main Street 1234 Main Street\nAshland, KY 41101 Ashland,  41101\nCCooddee DDeessccrriippttiioonn QTY Price Line Total\nSKU1222 Test Import Name - Description Goes Here 1 $10.00 $10.00\nLabor - Example labor item. Quantity is number of hours spent,  1.5 $100.00 $150.00\nprice is hourly rate. Quantity accepts decimal values.\nNotes\nAn invoice note can go here. Multi-line and even multi-page notes are supported.\nPPaayymmeenntt  DDeettaaiillss\nSubtotal $160.00\nShipping$10.00 Tax $0.78\nUPS Ground Total $170.78\nPayments (-) $0.00\nBalance Due $170.78\nAn invoice footer can go here'

In [5]:
print(text)

INVOICE
Invoice No. I1083
Account # C1006
Date 08-14-2008
Due By 08-31-2008
Demo Company
Phone : 111-222-3333 Terms None
1234 Main Street E-Mail : 333-444-4444 PO No. PO1234
Ashland, KY 41102 Web : http://www.ksoftware.net Sales Rep SalesPerson1
Bill To Ship To
Test Customer Test Customer
1234 Main Street 1234 Main Street
Ashland, KY 41101 Ashland,  41101
CCooddee DDeessccrriippttiioonn QTY Price Line Total
SKU1222 Test Import Name - Description Goes Here 1 $10.00 $10.00
Labor - Example labor item. Quantity is number of hours spent,  1.5 $100.00 $150.00
price is hourly rate. Quantity accepts decimal values.
Notes
An invoice note can go here. Multi-line and even multi-page notes are supported.
PPaayymmeenntt  DDeettaaiillss
Subtotal $160.00
Shipping$10.00 Tax $0.78
UPS Ground Total $170.78
Payments (-) $0.00
Balance Due $170.78
An invoice footer can go here


In [6]:
for row in text.split('\n'):
    if  row.startswith('Balance Due'):
        balance=row.split()[-1]

In [7]:
balance

'$170.78'

In [8]:
# for website using regex
import re
pattern = re.compile(r'https?://www\.?\w+\.\w+')
matches = pattern.findall(text)

for match in matches:
    print(match)

http://www.ksoftware.net


# -------------------------------------------------------------------

In [9]:
# Example 2 - a large semistructured invoice pdf

# Extract Invoice lines from a semistructured PDF page

In [10]:
import pdfplumber
import re
import pandas as pd

In [11]:
#Page 16 of the pdf


In [12]:
with pdfplumber.open('Dataset/apreports.pdf') as pdf:
    page = pdf.pages[15]
    text = page.extract_text()

In [13]:
print(text)

AccountsPayableReportPack
changeswillbereflectedunderthe"New"heading.Whenaninvoiceormanualcheckisdeleted,
theInvoiceAmountisautomaticallychangedtozeroandtheStatusfieldischangedto"D".
TotalTransactionCount Totalnumberoftransactionsonthelistincludingchangesanddeletions.Eachchangedtransaction
(i.e.,botholdandnew)countsasonetransaction.
1Notshownonsamplereport.
Invoice by Vendor List
Date: 08/25/2020 Invoice by Vendor List Page: 1
Jensen, Martin & Anderson, P.C.
Inv Due Inv Disc Disc Net
Voucher # Invoice # Date Date Amount Amount Date Amount 1099 Ref # Description Codes
200 United Parcel Service
200.01MLJ 3243387 032620 062620 75.00 P 75.00 1 Overnight document delivery * 1
3243387 032720 062720 75.00 P 75.00 3 Postage * 1
3243387 072620 072620 75.00 P 75.00 2 Overnight document delivery * 1
Vendor Totals 225.00 0.00 225.00
202 Software Technology, LLC
312205 082020 082520 490.00 P 490.00 1 Software Maintenance Agreement Tabs3 * 1
312206 082020 082520 239.00 P 239.00 2 Software Maintenanc

In [14]:
# Collecting vendor name
# We see vendor name starts with 3 digit number and text
# Use regex

new_vend_re = re.compile(r'^\d{3} [A-Z].*')


In [15]:
for line in text.split('\n'):
    if new_vend_re.match(line):
        print(line)

200 United Parcel Service
202 Software Technology, LLC
203 Clerk of the County Court
204 Clerk of the District Court
225 Jackson/Wylinda
240 NELCO
250 D & B Real Estate Management Company
300 Thomson Reuters Payment Center
325 Professional Messenger Services
400 Sprint Local & Long Distance
700 Clean All Janitorial Services


In [16]:
# separating vendor no. and vendor name from above
for line in text.split('\n'):
    if new_vend_re.match(line):
        vend_num, *vend_name = line.split()
        vend_name = ' '.join(vend_name)
        #print(vend_num, vend_name)
        print(vend_num)
        print(vend_name)

200
United Parcel Service
202
Software Technology, LLC
203
Clerk of the County Court
204
Clerk of the District Court
225
Jackson/Wylinda
240
NELCO
250
D & B Real Estate Management Company
300
Thomson Reuters Payment Center
325
Professional Messenger Services
400
Sprint Local & Long Distance
700
Clean All Janitorial Services


# Thinking process
Problems identified
1. Inconsistent
2. Some have voucher and invoice number, others don't  
3. Invoice might clash with Regex used here for dates (6digits)

Steps:
1. ignore voucher number and invoice number for now
2. consistent -- 6 digit date | space | 6 digit date | Inv.amt | 'P' letter or not | net.amt | Description


In [17]:
# getting regex right is the trickiest part

inv_line_re = re.compile(r'(\d{6}) (\d{6}) ([\d,]+\.\d{2})[\sP]*([\d,]+\.\d{2})[YN ]*\d\s?(.*?) [*\s\d]')
for line in text.split('\n'):
    if inv_line_re.search(line):
        print(line)                          #prints entire line with the matches given

200.01MLJ 3243387 032620 062620 75.00 P 75.00 1 Overnight document delivery * 1
3243387 032720 062720 75.00 P 75.00 3 Postage * 1
3243387 072620 072620 75.00 P 75.00 2 Overnight document delivery * 1
312205 082020 082520 490.00 P 490.00 1 Software Maintenance Agreement Tabs3 * 1
312206 082020 082520 239.00 P 239.00 2 Software Maintenance Agreement Tabs3 General Ledger * 1
312207 082020 082520 239.00 P 239.00 3 Software Maintenance Agreement Tabs3 Accounts Payable * 1
312208 082020 082520 239.00 P 239.00 4 Software Maintenance Agreement Tabs3 Trust Accounting * 1
312209 082020 082520 800.00 P 800.00 5 Software Maintenance Agreement PracticeMaster (9 Users) * 1
312210 082020 082520 105.00 P 105.00 6 Software Maintenance Agreement Tabs3 Taskbill * 1
320559 082520 083020 205.00 205.00 7 Purchase Tabs3 Device Interface Program 1
200.01PAM 042620 042620 225.00 P 225.00 1 Documentation Fee, Filing Fee, and Sheriff Service Fee * 1
032720 032720 75.00 P 75.00 2 Filing fee * 1
052020 052020 75.0

In [18]:
#Checking for Description part being correct
for line in text.split('\n'):
    line = inv_line_re.search(line)
    if line:
        print(line.group(5))

Overnight document delivery
Postage
Overnight document delivery
Software Maintenance Agreement Tabs3
Software Maintenance Agreement Tabs3 General Ledger
Software Maintenance Agreement Tabs3 Accounts Payable
Software Maintenance Agreement Tabs3 Trust Accounting
Software Maintenance Agreement PracticeMaster (9 Users)
Software Maintenance Agreement Tabs3 Taskbill
Purchase Tabs3 Device Interface Program
Documentation Fee, Filing Fee, and Sheriff Service Fee
Filing fee
Filing fee
Petition for Dissolution of Marriage Filing Fee
Holiday Party
Catering/Firm Retreat
Check order for APS and TAS Checks
Monthly Rent (Including Electric, Gas & Garbage)
Monthly Rent (Including Electric, Gas & Garbage)
Duplicate Key for building
Online Legal Research
Online Legal Research
Online Legal Research
Online Legal Research
Online Legal Research
Online Legal Research
Online Legal Research
Online Legal Research
Courier Fee
Courier Fee
Courier Fee
Courier Fee
November Cell Phone Charges
Monthly Cleaning Charges

In [19]:
#If we want 'Inv. Date' which is first group
for line in text.split('\n'):
    line = inv_line_re.search(line)
    if line:
        print(line.group(1))

032620
032720
072620
082020
082020
082020
082020
082020
082020
082520
042620
032720
052020
082020
122419
072620
072620
072220
072620
081320
042520
052420
062520
072920
073020
073020
073020
073020
062120
062420
082520
082520
082020
072220
072620


In [20]:
# We have succesfully created group of Inv and Due dates, Inv.amt, Net amt, Description.
# Put them into columns and make a dataframe

from collections import namedtuple

In [21]:
Inv = namedtuple('Inv', 'vend_num vend_name Inv_date Due_date Inv_amt Net_amt Description')

In [22]:
line_items=[]
for line in text.split('\n'):
    if new_vend_re.match(line):
        vend_num, *vend_name = line.split()
        vend_name = ' '.join(vend_name)
        
    line = inv_line_re.search(line)
    if line:
        inv_dt = line.group(1)
        due_dt = line.group(2)
        inv_amt = line.group(3)
        net_amt = line.group(4)
        desc = line.group(5)
        line_items.append(Inv(vend_num, vend_name, inv_dt, due_dt, inv_amt, net_amt, desc))
            
           

In [23]:
#check
line_items[:3]

[Inv(vend_num='200', vend_name='United Parcel Service', Inv_date='032620', Due_date='062620', Inv_amt='75.00', Net_amt='75.00', Description='Overnight document delivery'),
 Inv(vend_num='200', vend_name='United Parcel Service', Inv_date='032720', Due_date='062720', Inv_amt='75.00', Net_amt='75.00', Description='Postage'),
 Inv(vend_num='200', vend_name='United Parcel Service', Inv_date='072620', Due_date='072620', Inv_amt='75.00', Net_amt='75.00', Description='Overnight document delivery')]

In [24]:
#making it a Dataframe
df = pd.DataFrame(line_items)

In [25]:
df.head()

,vend_num,vend_name,Inv_date,Due_date,Inv_amt,Net_amt,Description
0,200,United Parcel Service,032620,062620,75.00,75.00,Overnight document delivery
1,200,United Parcel Service,032720,062720,75.00,75.00,Postage
2,200,United Parcel Service,072620,072620,75.00,75.00,Overnight document delivery
3,202,"Software Technology, LLC",082020,082520,490.00,490.00,Software Maintenance Agreement Tabs3
4,202,"Software Technology, LLC",082020,082520,239.00,239.00,Software Maintenance Agreement Tabs3 General L...


In [26]:
# convert date columns into datetime format
df['Inv_date'] = pd.to_datetime(df['Inv_date'])
df['Due_date'] = pd.to_datetime(df['Due_date'])

In [27]:
df.head()

,vend_num,vend_name,Inv_date,Due_date,Inv_amt,Net_amt,Description
0,200,United Parcel Service,2020-03-26,2020-06-26,75.00,75.00,Overnight document delivery
1,200,United Parcel Service,2020-03-27,2020-06-27,75.00,75.00,Postage
2,200,United Parcel Service,2020-07-26,2020-07-26,75.00,75.00,Overnight document delivery
3,202,"Software Technology, LLC",2020-08-20,2020-08-25,490.00,490.00,Software Maintenance Agreement Tabs3
4,202,"Software Technology, LLC",2020-08-20,2020-08-25,239.00,239.00,Software Maintenance Agreement Tabs3 General L...


In [28]:
# converting amount values string to float  (it has commas also)
df['Inv_amt'] = df['Inv_amt'].map(lambda x: float(x.replace(',','')))
df['Net_amt'] = df['Net_amt'].map(lambda x: float(x.replace(',','')))

In [29]:
df[['Inv_amt','Net_amt']].sum()

Inv_amt    22476.31
Net_amt    22476.31
dtype: float64

In [30]:
# It matches with the value shown in pdf at the bottom 'Grand Totals' (looks good)

In [32]:
# Exporting as csv
df.to_csv('Invoice1.csv')

# --------- End of '_Extract Invoice lines from a semistructured PDF page_' --------